# Download pictures from bing

In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, scrolledtext
from bing_image_downloader import downloader
import threading

## GUI:Graphical User Interface 图形用户接口

tkinter: Python的标准GUI库，用于创建图形用户界面。

ttk: 主题控件库（ttk）是 tkinter 的扩展模块，提供了一组现代化的控件，具有更好的外观和一致性。ttk 是 themed tk 的缩写，它通过使用操作系统的本地主题来改进 tkinter 控件的外观，使得应用程序看起来更现代和专业。

filedialog：是 tkinter 提供的一个模块，用于创建文件对话框。文件对话框允许用户在文件系统中选择文件或目录。filedialog 提供了多种方法来打开文件、保存文件以及选择目录。

scrolledtext:是 tkinter 提供的一个模块，用于创建带有滚动条的文本控件。它是 tkinter 的 Text 控件的扩展，自动添加了垂直滚动条，适合用于显示较长的文本内容，如日志信息。


In [ ]:
class ImageDownloaderGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("图片下载器")
        self.root.geometry("600x400")
        
        # 创建主框架,ttk
        self.main_frame = ttk.Frame(root, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 单个关键词下载部分,ttk
        ttk.Label(self.main_frame, text="输入关键词:").grid(row=0, column=0, sticky=tk.W)
        self.keyword_entry = ttk.Entry(self.main_frame, width=40)
        self.keyword_entry.grid(row=0, column=1, padx=5)
        ttk.Button(self.main_frame, text="下载", command=self.download_single).grid(row=0, column=2)
        
        # 添加图片数量控制,ttk
        ttk.Label(self.main_frame, text="下载数量:").grid(row=1, column=0, sticky=tk.W)
        self.limit_var = tk.StringVar(value="15")
        self.limit_entry = ttk.Entry(self.main_frame, width=10, textvariable=self.limit_var)
        self.limit_entry.grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
        
        # 文件批量下载部分,ttl
        ttk.Label(self.main_frame, text="或者选择关键词文件:").grid(row=2, column=0, sticky=tk.W, pady=10)
        ttk.Button(self.main_frame, text="选择文件", command=self.select_file).grid(row=2, column=1, sticky=tk.W)
        ttk.Button(self.main_frame, text="批量下载", command=self.download_batch).grid(row=2, column=2)
        
        # 日志显示区域
        self.log_area = scrolledtext.ScrolledText(self.main_frame, width=60, height=20)
        self.log_area.grid(row=3, column=0, columnspan=3, pady=10)
        
        self.keywords_file = None

    def log_message(self, message):
        self.log_area.insert(tk.END, message + "\n")
        self.log_area.see(tk.END)

    def get_download_limit(self):
        try:
            limit = int(self.limit_var.get())
            if limit <= 0:
                self.log_message("下载数量必须大于0，将使用默认值15")
                return 15
            return limit
        except ValueError:
            self.log_message("下载数量必须是数字，将使用默认值15")
            return 15

    def download_images(self, query):
        self.log_message(f"正在下载关键词 '{query}' 的图片...")
        try:
            downloader.download(
                query,
                limit=self.get_download_limit(),  # 使用用户输入的数量
                output_dir='downloads',
                adult_filter_off=True,
                force_replace=False,
                timeout=60,
                verbose=True
            )
            self.log_message(f"'{query}' 下载完成")
        except Exception as e:
            self.log_message(f"下载 '{query}' 时出错: {str(e)}")

    def download_single(self):
        keyword = self.keyword_entry.get().strip()
        if keyword:
            threading.Thread(target=self.download_images, args=(keyword,), daemon=True).start()
        else:
            self.log_message("请输入关键词")

    def select_file(self):
        self.keywords_file = filedialog.askopenfilename(
            filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
        )
        if self.keywords_file:
            self.log_message(f"已选择文件: {self.keywords_file}")

    def download_batch(self):
        if not self.keywords_file:
            self.log_message("请先选择关键词文件")
            return
            
        def batch_download():
            try:
                with open(self.keywords_file, 'r', encoding='utf-8') as f:
                    keywords = [line.strip() for line in f if line.strip()]
                
                for keyword in keywords:
                    self.download_images(keyword)
                
                self.log_message("批量下载完成")
            except Exception as e:
                self.log_message(f"批量下载出错: {str(e)}")
        
        threading.Thread(target=batch_download, daemon=True).start()

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageDownloaderGUI(root)
    root.mainloop()

# threading
threading：Python的线程模块，用于并发执行任务。是 Python 的标准库模块，用于创建和管理线程。线程允许程序并发执行多个任务，从而提高程序的响应速度和性能。在你的代码中，threading 模块用于在后台执行图片下载任务，以避免阻塞主线程，从而保持 GUI 的响应性。
